In [23]:
import os

In [24]:
%pwd


's:\\study\\data scientist\\projects\\End-To-End MLOps\\MLOps'

In [6]:
os.chdir("../")

In [44]:
%pwd

's:\\study\\data scientist\\projects\\End-To-End MLOps\\MLOps'

In [121]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [122]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories



In [123]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH, 
                 scheme_filepath = SCHEMA_FILE_PATH):
        

        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))
        self.scheme = read_yaml(str(scheme_filepath))

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
        

In [124]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [125]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")

        else:
            logger.info(f"File already exists at {self.config.local_data_file}.\n")
            logger.info(f"File size: {get_size(self.config.local_data_file)} bytes.")

    def extract_zip_file(self):
        
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

In [126]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:  
    logger.exception(e)
    raise e

[2025-06-06 21:14:14,541: INFO: common: YAML file config\config.yaml loaded successfully.]
[2025-06-06 21:14:14,548: INFO: common: YAML file params.yaml loaded successfully.]
[2025-06-06 21:14:14,549: INFO: common: YAML file schema.yaml loaded successfully.]
[2025-06-06 21:14:14,552: INFO: common: Created directory at: artifacts]
[2025-06-06 21:14:14,553: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-06-06 21:14:15,203: INFO: 1308545153: artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: CE86:213AEB:56E89F:5F4895:68433E06
Accept-Ranges: bytes
Date: Fri, 